In [68]:
#Import modules and dependecies
import pandas as pd
import numpy as np
import plotly.express as px

import requests

from dash import html, Dash, dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash_bootstrap_templates import load_figure_template

In [93]:
#Load dataframes
df1 = pd.read_csv('datasets/final_df1.csv')
df1_multi = pd.read_csv('datasets/final_df1.csv', index_col=[0,1])
df2 = pd.read_csv('datasets/final_df2.csv')

In [61]:
#Load the url for the GEO json file
repo_url = 'https://raw.githubusercontent.com/angelnmara/geojson/master/mexicoHigh.json' 
regiones_mexico = requests.get(repo_url).json()

In [73]:
df1.head(2)

,Estado,Year,Edad mediana,Población de 0 a 4 años,Población de 10 a 14 años,Población de 100 años y más,Población de 15 a 19 años,Población de 20 a 24 años,Población de 25 a 29 años,Población de 30 a 34 años,...,Población de 80 a 84 años,Población de 85 a 89 años,Población de 90 a 94 años,Población de 95 a 99 años,Población total,Porcentaje de 0-19 años,Porcentaje de 20-39 años,Porcentaje de 40-59 años,Porcentaje de 60-79 años,Porcentaje de 80 y más años
0,Aguascalientes,1995,19,112429,104083,59,97061,88901,71839,62217,...,3892,2428,975,380,862720,48.828473,31.934579,13.211703,4.983425,0.896467
1,Aguascalientes,2000,21,116137,108532,102,99577,87508,79200,70317,...,4336,2726,1169,542,944285,46.767766,31.543867,14.741524,5.292788,0.939865


In [85]:
#Create the map

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE, dbc_css])

load_figure_template('SLATE')

app.layout = html.Div([
    dbc.Row([
        html.H3(id='titulo-mapa', style={"text-align": "center"})
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Card(dcc.Markdown('Rango de edad:')),
            dcc.Dropdown(
                id='dropdown-edad',
                options=[
                    {'label': '0-19 años', 'value': 'Porcentaje de 0-19 años'},
                    {'label': '20-39 años', 'value': 'Porcentaje de 20-39 años'},
                    {'label': '40-59 años', 'value': 'Porcentaje de 40-59 años'},
                    {'label': '60-79 años', 'value': 'Porcentaje de 60-79 años'},
                    {'label': '+80 años', 'value': 'Porcentaje de 80 y más años'},
                ],
                value='Porcentaje de 80 y más años',
                className='dbc'
            ),
            html.Br(),
            dbc.Card(dcc.Markdown('Año de censo:')),
            dcc.Dropdown(
                id='dropdown-año',
                options=[1995,2000,2005,2010,2020],
                value=1995,
                className='dbc'
            )
        ],width=3),
        dbc.Col([
            dcc.Graph(id='mapa')
        ],width=8),
    ])
])

@app.callback(
    Output('titulo-mapa','children'),
    Output('mapa', 'figure'),
    Input('dropdown-edad', 'value'),
    Input('dropdown-año', 'value')
)
def mapa_estados(rango_edades, año_censo):
    df = df1.query('Year == @año_censo')
    
    title = f'Mapa de densidad de {rango_edades} del {año_censo} por Estado'
    
    fig = px.choropleth(
        df,
        geojson=regiones_mexico,
        locations=df['Estado'],
        featureidkey='properties.name',
        color=df[rango_edades],
        color_continuous_scale='Plasma'
    )
    fig.update_geos(
        showcountries=True, 
        showcoastlines=True, 
        showland=True, 
        fitbounds="locations"
    )
    
    return title, fig

app.run(
    debug=True, 
    port=56666, 
    jupyter_mode='external'
)

Dash app running on http://127.0.0.1:56666/


In [113]:
df2

,year,Tasa fertilidad,Población 65 años y más,Crecimiento poblacional (% anual),Población total,Población rural,Población urbana,Defunciones,Nacimientos,Matrimonios,Divorcios,Homicidios total,Suicidios
0,1994,3.127,4.386815,1.904435,88314424,23858142,64456282,419074,2904389,671640,35029,15839,2603
1,1995,3.038,4.471475,1.856808,89969572,23960696,66008876,430278,2750444,658114,37455,15612,2894
2,1996,2.956,4.565473,1.781296,91586555,24114740,67471815,436321,2707718,670523,38545,14505,3020
3,1997,2.869,4.668195,1.728183,93183094,24293764,68889330,440437,2698425,707840,40792,13552,3370
4,1998,2.800,4.776700,1.685793,94767284,24463227,70304057,444665,2668428,704456,45889,13656,3342
5,1999,2.766,4.889968,1.640548,96334810,24622214,71712596,443950,2769089,743856,49271,12249,3339
6,2000,2.716,5.004304,1.584551,97873442,24740449,73132993,437667,2798339,707422,52358,10737,3475
7,2001,2.669,5.125690,1.541941,99394288,24803845,74590443,443127,2767610,665434,57370,10285,3811
8,2002,2.623,5.254809,1.520455,100917081,24860923,76056158,459687,2699084,616654,60641,10088,3871
9,2003,2.579,5.382524,1.487401,102429341,24908767,77520574,472140,2655894,584142,64248,10087,4104


In [119]:
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE, dbc_css])

load_figure_template('SLATE')

app.layout = html.Div([
    dbc.Row([
        dbc.Card(
            html.H3(id='titulo-graph', style={"text-align": "center"}))
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Card(
                dcc.RadioItems(
                    id='radio-variables',
                    options=df2.columns[1:],
                    value='Población total',
                    className='dbc'
                )
            )
        ],width=3),
        dbc.Col([
            dbc.Card(
                dcc.Graph(id='grafico-mexico')
            )
        ])
    ])
])

@app.callback(
    Output('titulo-graph', 'children'),
    Output('grafico-mexico', 'figure'),
    Input('radio-variables', 'value')
)
def grafico_mexico(variable):
    fig = px.line(
        df2,
        x='year',
        y=variable
    )
    
    title = f'{variable} en México de 1994-2022'
    
    return title, fig
app.run(
    debug=True, 
    port=56333, 
    jupyter_mode='external'
)

Dash app running on http://127.0.0.1:56333/


In [125]:
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE, dbc_css])

load_figure_template('SLATE')

app.layout = dbc.Container([
    dbc.Tabs([
        dbc.Tab(label='Mapa por Rango de Edades', children=[
            html.Div([
                dbc.Row([
                    html.H3(id='titulo-mapa', style={"text-align": "center"})
                ]),
                dbc.Row([
                    dbc.Col([
                        dbc.Card(dcc.Markdown('Rango de edad:')),
                        dcc.Dropdown(
                            id='dropdown-edad',
                            options=[
                                {'label': '0-19 años', 'value': 'Porcentaje de 0-19 años'},
                                {'label': '20-39 años', 'value': 'Porcentaje de 20-39 años'},
                                {'label': '40-59 años', 'value': 'Porcentaje de 40-59 años'},
                                {'label': '60-79 años', 'value': 'Porcentaje de 60-79 años'},
                                {'label': '+80 años', 'value': 'Porcentaje de 80 y más años'},
                            ],
                            value='Porcentaje de 80 y más años',
                            className='dbc'
                        ),
                        html.Br(),
                        dbc.Card(dcc.Markdown('Año de censo:')),
                        dcc.Dropdown(
                            id='dropdown-año',
                            options=[1995,2000,2005,2010,2020],
                            value=1995,
                            className='dbc'
                        )
                    ],width=3),
                    dbc.Col([
                        dcc.Graph(id='mapa')
                    ],width=8),
                ])
            ])
        ]),
        dbc.Tab(label='México a través del tiempo', children=[
          html.Div([
                dbc.Row([
                    dbc.Card(
                        html.H3(id='titulo-graph', style={"text-align": "center"}))
                ]),
                dbc.Row([
                    dbc.Col([
                        dbc.Card(
                            dcc.RadioItems(
                                id='radio-variables',
                                options=df2.columns[1:],
                                value='Población total',
                                className='dbc'
                            )
                        )
                    ],width=3),
                    dbc.Col([
                        dbc.Card(
                            dcc.Graph(id='grafico-mexico')
                        )
                    ])
                ])
            ])  
        ]),
    ], className="dbc")
])

@app.callback(
    Output('titulo-mapa','children'),
    Output('mapa', 'figure'),
    Input('dropdown-edad', 'value'),
    Input('dropdown-año', 'value')
)
def mapa_estados(rango_edades, año_censo):
    df = df1.query('Year == @año_censo')
    
    title = f'Mapa de densidad de {rango_edades} del {año_censo} por Estado'
    
    fig = px.choropleth(
        df,
        geojson=regiones_mexico,
        locations=df['Estado'],
        featureidkey='properties.name',
        color=df[rango_edades],
        color_continuous_scale='Plasma'
    )
    fig.update_geos(
        showcountries=True, 
        showcoastlines=True, 
        showland=True, 
        fitbounds="locations"
    )
    
    return title, fig

@app.callback(
    Output('titulo-graph', 'children'),
    Output('grafico-mexico', 'figure'),
    Input('radio-variables', 'value')
)
def grafico_mexico(variable):
    fig = px.line(
        df2,
        x='year',
        y=variable
    )
    
    title = f'{variable} en México de 1994-2022'
    
    return title, fig

app.run(
    debug=True, 
    port=51234, 
    jupyter_mode='external'
)

Dash app running on http://127.0.0.1:51234/
